# QUANTILE T-TEST
ML инженеры во FlyingFood, сервисе по доставке еды с помощью дронов-курьеров, разработали модель для оценки времени доставки заказа – ETA (estimated time of arrival). Это время мы показываем клиенту в приложении, когда тот выбирает, какое блюдо заказать, а также во время самого ожидания доставки. 

Если мы обещали время доставки 20 минут, а дрон-курьер летел час – клиент слишком долго ждёт, чтобы его удержать, придётся тратить бюджет, давать купоны и скидки, чтобы сохранить его лояльность. Если даём слишком консервативные оценки, он может принять решение, что у нас долгая доставка, отменить доставку и заказать у конкурентов. Обе ошибки неприятные, но у недопредсказания цена ошибки выше. 

Ваши коллеги выкатили новую модель, которая, по их словам, ведёт себя лучше в худшем случае (сокращает большой недопрогноз). Мы провели A/B-эксперимент, вас подключили к его анализу, когда он закончился. Нетрудно заметить, что то, как улучшился прогноз в среднем, здесь нас не очень интересует, поэтому обычный t-тест для средних здесь не применим. 

Какой нужен статистический критерий для сравнения квантилей? 

Как сопоставить хвосты распределений в группах A и B? 

# Статистический критерий
В сердце любого A/B-теста – статистический критерий, который определяет, стала ли целевая метрика лучше или хуже. Метрика фиксируется до начала эксперимента. В нашем случае эта метрика – ошибка прогноза для каждого заказа.



Какие данные мы имеем на входе?

Контрольная выборка (группа A) — распределение ошибок прогноза для старого алгоритма.
Экспериментальная выборка (группа B) — распределение ошибок прогноза для нового алгоритма.
Популярный выбор — t-тест Стьюдента, который проверяет гипотезу о том, что средние значения в двух выборках не различаются. Готовую имплементацию t-теста можно найти в в библиотеке scipy. 

Как мы обсуждали ранее, для основной гипотезы, которую мы тестировали A/B-экспериментом, он не подходит, поскольку нас интересует изменение в хвостах распределений, а не в среднем. Однако стандартный t-тест – это именно то, что нам нужно, чтобы проверить наше базовое предположение, что новый алгоритм не повлиял на среднюю ошибку прогноза.

Иными словами, что не добавилось неожиданных сторонних эффектов.

# Задача
Ваша задача: написать функцию, которая проверяет, есть ли значимое различие средних ошибок прогнозов времени доставки в контрольной и экспериментальных выборках (в группах A и B). 

In [5]:
from typing import List, Tuple
from scipy import stats


def ttest(
    control: List[float],
    experiment: List[float],
    alpha: float = 0.05,
) -> Tuple[float, bool]:
    """Two-sample t-test for the means of two independent samples"""
    _, p_value = stats.ttest_ind(control, experiment)
    result = bool(p_value <= alpha)

    return p_value, result

In [6]:
# Пример данных для контрольной и экспериментальной групп
control_group = [20.1, 21.5, 22.3, 19.8, 20.9]
experiment_group = [22.4, 23.1, 21.9, 23.5, 24.1]

# Уровень значимости alpha
alpha = 0.05

# Вызов функции ttest
p_value, result = ttest(control_group, experiment_group, alpha)

# Вывод результатов
print(f"P-value: {p_value:.4f}")
# Вывод результатов
if result:
    print(f"Нет статистически значимой разницы (p > {alpha})")
else:
    print(f"Есть статистически значимая разница (p <= {alpha})")


P-value: 0.0085
Нет статистически значимой разницы (p > 0.05)


Как мы и предполагали, средние совпали.

Переходим к сравнению квантилей. Напомним, N%-ым квантилем (или, что то же самое, N-ый персентиль) – называют такое значение наблюдаемой величины, что ровно N% значений будет меньше.

В данном A/B-эксперименте нас интересовало, как изменится ошибка обещанного пользователю времени доставки в самых плохих сценариях: чем больше ошибка, тем интенсивнее её негативный эффект на поведение пользователя. 

Но вот незадача. У каждой из выборок (контрольной и тестовой группах) всего одно конкретное значение 95-го квантиля. В это время, чтобы применять статистические критерии и считать p-value, нужны выборки. Выборка из 1 элемента – не звучит как валидная опция. Ну стал 95-й квантиль в группе B меньше чем в группе A если на 3 минуты, достаточно ли этого для принятия решения? А если стал на 5 минут? на 10? или на 1.2 минуты? Где проходит порог достаточно большой разницы и недостаточно большой? Как нам перейти от сравнения точечных значений – обратно к сравнению выборок?

Если бы у нас была возможность провести 10 000 экспериментов, мы бы собрали 10 000 выборок и оценили в каждом эксперименте квантиль в группах A и в группах B. Применив стандартный t-тест мы бы поняли, есть ли эффект или нет.

Однако 10 000 экспериментов звучит как избыточно большое количество данных.

Можно ли сделать нечто подобное с результатами одного эксперимента?

Ответ: да, можем.

# Bootstrap
Бутстрэп (bootstrap) — это техника, которая позволяет симулировать проведение тех самых 10 000 экспериментов и перейти от точечной оценки любой статистики (в нашем случае, 95-го персентиля) – к её распределению. 

Для этого из исходной выборки размерности N случайным образом сэмплируетсся N элементов с повторениями. У такой искусственной выборки считается нужная статистика и сохраняется. Действие повторяется тысячи раз: каждый раз какие-то элементы то попадают в очередную искусственную выборку, а то и по несколько раз, то не попадают совсем.

В результате такого "шатания" мы переходим от точечной оценки статистики к её распределению, у которого можем посчитать доверительный интервал или к которому можем применить статистический тест.

Напомним, доверительный интервал – это диапазон, который включает значение неизвестной статистики с заданной вероятностью. Например, 95%-доверительный интервал для среднего времени доставки с вероятностью 95% включает реальное среднее время доставки.

Пример того, как можно посчитать распределение медианы с помощью бутстрэпа:

In [7]:
import numpy as np

def bootstrapped_median(x: List[float], n_bootstraps: int = 10_000) -> List[float]:
    """Bootstrapped median distribution"""
    bootstrapped_medians = []
    
    for _ in range(n_bootstraps):
        bootstrapped_sample = np.random.choice(x, size=len(x), replace=True)
        bootstrapped_medians.append(np.median(bootstrapped_sample))
        
    return bootstrapped_medians


После чего можно оценить 75-ый персентиль медианы с помощью:

In [8]:
# samples = bootstrapped_median(x, n_bootstraps=1000)
# q75 = sorted(bootstrapped_median(x))[750]

# Задача
Реализуйте bootstrapped t-тест для сравнения квантилей ошибок прогноза.

Формат выходных данных совпадает с прошлым шагом.

In [9]:
import numpy as np

def bootstrapped_mean(x: List[float], n_bootstraps: int = 1000) -> List[float]:
    """Bootstrapped median distribution"""
    bootstrapped_means = []
    
    for _ in range(n_bootstraps):
        bootstrapped_sample = np.random.choice(x, size=len(x), replace=True)
        bootstrapped_means.append(np.mean(bootstrapped_sample))
        
    return bootstrapped_means

In [11]:
from typing import List
from typing import Tuple

import numpy as np
from scipy.stats import ttest_ind


def quantile_ttest(
    control: List[float],
    experiment: List[float],
    alpha: float = 0.05,
    quantile: float = 0.95,
    n_bootstraps: int = 1000,
) -> Tuple[float, bool]:
    """
    Bootstrapped t-test for quantiles of two samples.
    """
    control_bootstraps = bootstrapped_mean(control, n_bootstraps)
    experiment_bootstraps = bootstrapped_mean(experiment, n_bootstraps)

    # Вычисляем квантиль для каждого бутстрепа
    control_q = np.quantile(control_bootstraps, quantile)
    experiment_q = np.quantile(experiment_bootstraps, quantile)

    # t-test для массивов бутстрепированных выборок
    _, p_value = ttest_ind(control_bootstraps, experiment_bootstraps)

    # Сравниваем p_value с уровнем значимости alpha
    result = bool(p_value <= alpha)
    return p_value, result


Для решения этой задачи вы обрели новые навыки по статистическому тестированию гипотез и A/B тестам в машинном обучении. Кроме того, вы узнали про использование bootstrap-метод для оценки доверительных интервалов и квантилей распределений. Эти навыки могут быть очень полезны при работе с данными и в машинном обучении! 